In [1]:
import os
import subprocess

# For processing
import pandas as pd
import numpy as np

# For plotting
import matplotlib.pyplot as plt
import seaborn as sns

# For data split
from sklearn.model_selection import train_test_split

# For Random Forest testing
import h2o

# For config parsing
import configparser
config = configparser.ConfigParser()
config.read_file(open('config/config-azure.ini'))

client_attributes_file_path = config['data-files']['clients_attributes']
active_promos_file_path = config['data-files']['active_promos']
executed_promos_file_path = config['data-files']['executed_promos']
sales_file_path = config['data-files']['sales']


In [2]:
sales = pd.read_csv(sales_file_path, encoding='latin-1')
sales

,Cliente,Año,Mes,ClaseEnvase,SegmentoPrecio,Marca,Cupo,Nr,Hl,Dcto
0,1,2018,1,2,2.0,44,10.0,6352.94,0.03,0.0
1,1,2018,1,2,4.0,18,16.0,99130.50,0.45,0.0
2,1,2018,6,2,1.0,2,6.0,11605.40,0.07,-2880.0
3,1,2018,7,1,2.0,37,34.0,1944.01,0.01,0.0
4,1,2018,8,2,1.0,2,16.0,16250.18,0.06,0.0
...,...,...,...,...,...,...,...,...,...,...
1894247,15068,2019,8,1,2.0,44,32.0,3777.28,0.01,0.0
1894248,15069,2019,3,1,2.0,44,32.0,6295.46,0.02,0.0
1894249,15069,2019,4,1,2.0,44,32.0,6295.46,0.02,0.0
1894250,15069,2019,4,2,1.0,2,6.0,15741.56,0.08,0.0


In [3]:
executed_promos = pd.read_csv(executed_promos_file_path, encoding='latin-1')
sanitized_executed_promos = executed_promos.drop_duplicates(subset = ['Cliente', 'Marca', 'Cupo'])
sanitized_executed_promos

,CodigoDC,Cliente,Marca,Cupo
0,297601,8410,29,9
1,297601,9337,29,9
2,309843,7404,40,16
3,309874,4017,40,16
4,309910,6316,40,16
...,...,...,...,...
27763,646815,2144,29,9
27764,646976,28,29,9
27765,647188,10023,29,9
27766,648099,10193,29,9


In [4]:
pd.read_csv(active_promos_file_path, encoding='latin-1')

,CodigoDC,Marca,Cupo,Fecha_Desde,Fecha_Hasta,Cliente
0,297601,29,9,2018-08-06,2018-08-30,3213
1,297601,29,9,2018-08-06,2018-08-30,3795
2,297601,29,9,2018-08-06,2018-08-30,11816
3,298856,40,16,2018-08-12,2018-08-19,8444
4,309916,40,16,2018-08-12,2018-08-19,8488
...,...,...,...,...,...,...
422432,688354,40,16,2019-09-25,2019-09-26,14435
422433,688355,29,9,2019-09-25,2019-09-26,320
422434,688355,29,9,2019-09-25,2019-09-26,3117
422435,688355,29,9,2019-09-25,2019-09-26,3503


In [5]:
clients_attributes = pd.read_csv(client_attributes_file_path, encoding='latin-1')
clients_attributes

,Cliente,FechaAltaCliente,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF
0,1,2009-03-03,4,21,12,1,1,1
1,9,2009-03-03,4,21,9,2,3,1
2,15,2009-03-03,4,21,9,2,3,0
3,37,2009-03-03,4,21,16,2,3,1
4,43,2009-03-03,4,21,9,2,3,0
...,...,...,...,...,...,...,...,...
15064,15040,2019-03-07,1,16,9,2,1,0
15065,15043,2019-03-07,1,20,15,2,2,1
15066,15052,2019-03-09,3,7,10,2,2,0
15067,15065,2019-03-16,4,21,9,2,2,0


In [6]:
sales_exec_promos = sales.merge(sanitized_executed_promos, how = 'left', on = ['Cliente', 'Marca', 'Cupo'])
sales_exec_promos.loc[sales_exec_promos['CodigoDC'].isnull() == False, 'ES_PROMO'] = 'SI'

sales_exec_promos

,Cliente,Año,Mes,ClaseEnvase,SegmentoPrecio,Marca,Cupo,Nr,Hl,Dcto,CodigoDC,ES_PROMO
0,1,2018,1,2,2.0,44,10.0,6352.94,0.03,0.0,NaN,NaN
1,1,2018,1,2,4.0,18,16.0,99130.50,0.45,0.0,NaN,NaN
2,1,2018,6,2,1.0,2,6.0,11605.40,0.07,-2880.0,NaN,NaN
3,1,2018,7,1,2.0,37,34.0,1944.01,0.01,0.0,NaN,NaN
4,1,2018,8,2,1.0,2,16.0,16250.18,0.06,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1894247,15068,2019,8,1,2.0,44,32.0,3777.28,0.01,0.0,NaN,NaN
1894248,15069,2019,3,1,2.0,44,32.0,6295.46,0.02,0.0,NaN,NaN
1894249,15069,2019,4,1,2.0,44,32.0,6295.46,0.02,0.0,NaN,NaN
1894250,15069,2019,4,2,1.0,2,6.0,15741.56,0.08,0.0,NaN,NaN


In [7]:
df = sales_exec_promos.merge(clients_attributes, how = 'left', on = 'Cliente')
final_df = df[['Marca', 'Cupo', 'Region', 'Gerencia', 'SubCanal', 'TipoPoblacion', 'Estrato', 'EF', 'ES_PROMO']].fillna({
    'ES_PROMO': 'NO',
    'Cupo': df['Cupo'].mode()[0]
})

final_df

,Marca,Cupo,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,ES_PROMO
0,44,10.0,4,21,12,1,1,1,NO
1,18,16.0,4,21,12,1,1,1,NO
2,2,6.0,4,21,12,1,1,1,NO
3,37,34.0,4,21,12,1,1,1,NO
4,2,16.0,4,21,12,1,1,1,NO
...,...,...,...,...,...,...,...,...,...
1894247,44,32.0,1,5,9,2,3,1,NO
1894248,44,32.0,4,14,9,2,1,0,NO
1894249,44,32.0,4,14,9,2,1,0,NO
1894250,2,6.0,4,14,9,2,1,0,NO


### Preparar predictores y Target

In [8]:
predictor_cols = ['Marca', 'Cupo', 'Region', 'Gerencia', 'SubCanal', 'TipoPoblacion', 'Estrato', 'EF']
target_col     = 'ES_PROMO'

# predictors = final_df[predictor_cols]
# target     = final_df[target_col]

### Preparar Data de Test y Train

In [9]:
h2o.init(nthreads = -1, max_mem_size = '16g')

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.9" 2020-10-20; OpenJDK Runtime Environment (build 11.0.9+11-Ubuntu-0ubuntu1.18.04.1); OpenJDK 64-Bit Server VM (build 11.0.9+11-Ubuntu-0ubuntu1.18.04.1, mixed mode, sharing)
  Starting server from /anaconda/envs/py37_default/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmphfz629c3
  JVM stdout: /tmp/tmphfz629c3/h2o_tjander_started_from_python.out
  JVM stderr: /tmp/tmphfz629c3/h2o_tjander_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.2
H2O_cluster_version_age:,11 days
H2O_cluster_name:,H2O_from_python_tjander_92dk7u
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,16 Gb
H2O_cluster_total_cores:,6
H2O_cluster_allowed_cores:,6
H2O_cluster_status:,"accepting new members, healthy"


In [10]:
h2o_df = h2o.H2OFrame(final_df)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [11]:
m = h2o.estimators.H2ORandomForestEstimator(nfolds=10, #CV- las particiones de los datos
                                            max_depth=20,#profundidad máxima de los árboles
                                            ntrees=50, #número de árboles
                                            seed=2020) #semilla

In [ ]:
m.train(training_frame = h2o_df,
        x = predictor_cols,
        y = target_col)

drf Model Build progress: |█

In [ ]:
m.varimp_plot()

In [ ]:
performance = m.model_performance()
type(performance)

In [ ]:
performance.plot(type="roc")